<a id='building-your-own-algo-container'></a>

# Building your own TensorFlow algorithm container for SageMaker Training

With Amazon SageMaker, you can package your own algorithms that can then be trained and deployed in the SageMaker environment. This notebook guides you through an example using TensorFlow that shows you how to build a Docker container for SageMaker and use it for training.

By packaging an algorithm in a container, you can bring almost any code to the Amazon SageMaker environment, regardless of programming language, environment, framework, or dependencies. 


1. [Building your own algorithm container](#building-your-own-algo-container)
  1. [When should I build my own algorithm container?](#when-should-i)
  1. [Permissions](#permissions)
  1. [The Example](#the-example)
  1. [The Lab](#the-lab)
1. [Part 1: Packaging and Uploading your Algorithm for use with Amazon SageMaker](#part1)
    1. [An overview of Docker](#docker-overview)
    1. [How Amazon SageMaker runs your Docker container](#how-sagemaker-runs-docker)
    1. [The parts of the sample container](#parts-of-container)
    1. [The Dockerfile](#the-dockerfile)
    1. [Building and registering the container](#building-the-container)
1. [Part 2: Using your Algorithm in Amazon SageMaker](#part2)
  1. [Set up the environment](#set-up-environment)
  1. [Create the session](#create-the-session)
  1. [Upload the data for training](#upload-data-for-training)
  1. [Create an estimator and fit the model](#create-and-fit-estimator)

<a id='when-should-i'></a>

## When should I build my own algorithm container?

You may not need to create a container to bring your own code to Amazon SageMaker. When you are using a framework (such as Apache MXNet or TensorFlow) that has direct support in SageMaker, you can simply supply the Python code that implements your algorithm using the SDK entry points for that framework. This set of frameworks is continually expanding, so we recommend that you check the current list if your algorithm is written in a common machine learning environment.

Even if there is direct SDK support for your environment or framework, you may find it more effective to build your own container. If the code that implements your algorithm is quite complex on its own or you need special additions to the framework, building your own container may be the right choice.

If there isn't direct SDK support for your environment, don't worry. You'll see in this lab that building your own container is quite straightforward and can easily be automated.

<a id='permissions'></a>

## Permissions

### Modifying your SageMaker role

Running this notebook requires permissions in addition to the normal `SageMakerFullAccess` permissions. These additional permissions are required in order to allow SageMaker to:

* Create new repositories in Amazon Elastic Container Registry (ECR), which is used to host Docker images in AWS
* Create and build a new project in AWS CodeBuild (ie: to build our custom Docker image)

The easiest way to add these permissions is simply to add the managed policies `AmazonEC2ContainerRegistryFullAccess` and `AWSCodeBuildAdminAccess` to the Execution Role that is associated with your SageMaker Studio user. There's no need to restart your notebook instance when you do this - the new permissions will be available immediately.

To add these 2 required managed policies to your existing SageMaker Execution Role:

* Open a new instance of the AWS Console in a separate browser tab/window
* On the left-hand side, choose `Roles`
* Search for your execution role (usually something like AmazonSageMaker-ExecutionRole-...)
* Click the correct role in the list, and choose `Attach Policies`
* Search for the `AmazonEC2ContainerRegistryFullAccess` and `AWSCodeBuildAdminAccess` policies, placing a checkmark next to each
* Click `Attach Policy`

### Creating a CodeBuild role

This lab uses AWS CodeBuild to automatically build our Docker container image, and then push the new image into ECR so that SageMaker can use the image for training. In order to leverage CodeBuild, you will first need to create a new service-linked role in your account called `codebuild__sagemaker_workshop_role`. The role will require the following inline policy:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "CloudWatchLogsPolicy",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Sid": "S3GetPutPolicy",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:GetObjectVersion",
                "s3:PutObject",
                "s3:GetBucketAcl",
                "s3:GetBucketLocation"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Sid": "ECRPushPullPolicy",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken",
                "ecr:BatchCheckLayerAvailability",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage",
                "ecr:InitiateLayerUpload",
                "ecr:UploadLayerPart",
                "ecr:CompleteLayerUpload",
                "ecr:PutImage"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}
```

To create the new service-linked role for CodeBuild:

* Open a new instance of the AWS Console in a separate browser tab/window
* On the left-hand side, choose `Policies`
* Click `Create Policy`
* Click the `JSON` tab at the top of the page
* Copy the above JSON policy, and paste it into the JSON editor, overwriting the existing contents of the editor
* Click `Review Policy`
* In the `Name` field, enter `codebuild__sagemaker_workshop_policy` exactly as you see it here
* Click `Create Policy`
* On the left-hand side of the main IAM service screen, choose `Roles`
* Click `Create Role`
* At the top of the page, ensure that `AWS Service` is highlighted. Next, scroll to the bottom of the page and click on `CodeBuild`
* Click the `Next: Permissions` button
* Search for the `codebuild__sagemaker_workshop_policy` policy, and place a checkmark next to it. 
* Click `Next: Tags` then `Next: Review`
* In the `Role name` field, enter `codebuild__sagemaker_workshop_role` exactly as you see it here
* Click `Create role`


### Allowing SageMaker to pass the CodeBuild service-linked role to CodeBuild

Lastly, we need to adjust your SageMaker Execution Role so that SageMaker has permission to pass the appropriate CodeBuild role into CodeBuild when submitting CodeBuild jobs.

Begin by running the following Jupyter cell to generate an inline policy that is specific to your account:

In [ ]:
from IPython.display import display, Markdown
import json
import sagemaker

sess = sagemaker.session.Session()
account = sess.boto_session.client('sts').get_caller_identity()['Account']

policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:passRole",
            "Resource": "arn:aws:iam::" + account + ":role/codebuild__sagemaker_workshop_role"
        }
    ]
}

display(Markdown("#### Generated Inline Policy:"))
display(Markdown("```json\n" + json.dumps(policy, indent=4) + "\n```"))


Next, return to the IAM service console, and add the above generated policy to your existing SageMaker Execution Role as an 'inline policy':

* Open a new instance of the AWS Console in a separate browser tab/window
* On the left-hand side, choose `Roles`
* Search for your execution role (usually something like AmazonSageMaker-ExecutionRole-...)
* Click `Add Inline Policy` on the right-hand side
* Click the `JSON` tab at the top of the screen
* Paste your copied inline policy (from above) into the JSON editor, replacing all existing contents
* Click `Review Policy`
* In the `Name` field, type `sagemaker_workshop__codebuild_passrole`
* Click `Create Policy`

<a id='the-example'></a>

## The Example

In this example we show how to package a custom TensorFlow container with a Python training script which works with the CIFAR-10 dataset and uses TensorFlow Serving for inference. However, different inference solutions other than TensorFlow Serving can be used by modifying the docker container.

In this example, we use a single image to support training and hosting. This simplifies the procedure because we only need to manage one image for both tasks. Sometimes you may want separate images for training and hosting because they have different requirements. In this case, separate the parts discussed below into separate Dockerfiles and build two images. Choosing whether to use a single image or two images is a matter of what is most convenient for you to develop and manage.

If you're only using Amazon SageMaker for training or hosting, but not both, only the functionality used needs to be built into your container.

[CIFAR-10]: http://www.cs.toronto.edu/~kriz/cifar.html

<a id='the-lab'></a>

## The Lab

This Lab is divided into two parts: _Part1: building the container_ and _Part 2: using the container_.

<br/>
<br/>

<a id='part1'></a>

# Part 1: Packaging and Uploading your Algorithm for use with Amazon SageMaker

<a id='docker-overview'></a>

### An overview of Docker

For many data scientists, Docker containers are a new technology. But they are not difficult and can significantly simply the deployment of your software packages. 

Docker provides a simple way to package arbitrary code into an _image_ that is totally self-contained. Once you have an image, you can use Docker to run a _container_ based on that image. Running a container is just like running a program on the machine except that the container creates a fully self-contained environment for the program to run. Containers are isolated from each other and from the host environment, so the way your program is set up is the way it runs, no matter where you run it.

Docker is more powerful than environment managers like conda or virtualenv because (a) it is completely language independent and (b) it comprises your whole operating environment, including startup commands, and environment variable.

A Docker container is like a virtual machine, but it is much lighter weight. For example, a program running in a container can start in less than a second and many containers can run simultaneously on the same physical or virtual machine instance.

Docker uses a simple file called a `Dockerfile` to specify how the image is assembled. An example is provided below. You can build your Docker images based on Docker images built by yourself or by others, which can simplify things quite a bit.

Docker has become very popular in programming and devops communities due to its flexibility and its well-defined specification of how code can be run in its containers. It is the underpinning of many services built in the past few years, such as [Amazon ECS].

Amazon SageMaker uses Docker to allow users to train and deploy arbitrary algorithms.

In Amazon SageMaker, Docker containers are invoked in a one way for training and another, slightly different, way for hosting. The following sections outline how to build containers for the SageMaker environment.

Some helpful links:

* [Docker home page](http://www.docker.com)
* [Getting started with Docker](https://docs.docker.com/get-started/)
* [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
* [`docker run` reference](https://docs.docker.com/engine/reference/run/)

[Amazon ECS]: https://aws.amazon.com/ecs/

<a id='how-sagemaker-runs-docker'></a>

### How Amazon SageMaker runs your Docker container

Because you can run the same image in training or hosting, Amazon SageMaker runs your container with the argument `train` or `serve`. How your container processes this argument depends on the container.

* In this example, we don't define an `ENTRYPOINT` in the Dockerfile so Docker runs the command [`train` at training time](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) and [`serve` at serving time](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html). In this example, we define these as executable Python scripts, but they could be any program that we want to start in that environment.
* If you specify a program as an `ENTRYPOINT` in the Dockerfile, that program will be run at startup and its first argument will be `train` or `serve`. The program can then look at that argument and decide what to do.
* If you are building separate containers for training and hosting (or building only for one or the other), you can define a program as an `ENTRYPOINT` in the Dockerfile and ignore (or verify) the first argument passed in. 

#### Running your container during training

When Amazon SageMaker runs training, your `train` script is run, as in a regular Python program. A number of files are laid out for your use, under the `/opt/ml` directory:

    /opt/ml
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure

##### The input

* `/opt/ml/input/config` contains information to control how your program runs. `hyperparameters.json` is a JSON-formatted dictionary of hyperparameter names to values. These values are always strings, so you may need to convert them. `resourceConfig.json` is a JSON-formatted file that describes the network layout used for distributed training.
* `/opt/ml/input/data/<channel_name>/` (for File mode) contains the input data for that channel. The channels are created based on the call to CreateTrainingJob but it's generally important that channels match algorithm expectations. The files for each channel are copied from S3 to this directory, preserving the tree structure indicated by the S3 key structure. 
* `/opt/ml/input/data/<channel_name>_<epoch_number>` (for Pipe mode) is the pipe for a given epoch. Epochs start at zero and go up by one each time you read them. There is no limit to the number of epochs that you can run, but you must close each pipe before reading the next epoch.

##### The output

* `/opt/ml/model/` is the directory where you write the model that your algorithm generates. Your model can be in any format that you want. It can be a single file or a whole directory tree. SageMaker packages any files in this directory into a compressed tar archive file. This file is made available at the S3 location returned in the `DescribeTrainingJob` result.
* `/opt/ml/output` is a directory where the algorithm can write a file `failure` that describes why the job failed. The contents of this file are returned in the `FailureReason` field of the `DescribeTrainingJob` result. For jobs that succeed, there is no reason to write this file as it is ignored.

#### Running your container during hosting

Hosting has a very different model than training because hosting is reponding to inference requests that come in via HTTP. In this example, we use [TensorFlow Serving](https://www.tensorflow.org/serving/), however the hosting solution can be customized. One example is the [Python serving stack within the scikit learn example](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb).

Amazon SageMaker uses two URLs in the container:

* `/ping` receives `GET` requests from the infrastructure. Your program returns 200 if the container is up and accepting requests.
* `/invocations` is the endpoint that receives client inference `POST` requests. The format of the request and the response is up to the algorithm. If the client supplied `ContentType` and `Accept` headers, these are passed in as well. 

The container has the model files in the same place that they were written to during training:

    /opt/ml
    `-- model
        `-- <model files>



<a id='parts-of-container'></a>

### The parts of the sample container

The `container` directory has all the components you need to package the sample algorithm for Amazon SageMager:

    .
    |-- Dockerfile
    |-- build_and_push.sh
    |-- buildspec.yml
    `-- cifar10
        |-- cifar10.py
        |-- resnet_model.py
        |-- nginx.conf
        |-- serve
        `-- train

Let's discuss each of these in turn:

* __`Dockerfile`__ describes how to build your Docker container image. More details are provided below.
* __`build_and_push.sh`__ is a script that uses the Dockerfile to build your container images and then pushes it to ECR. We invoke the commands directly later in this notebook, but you can just copy and run the script for your own algorithms.
* __`buildspec.yml`__ describes the build process that will be used by AWS CodeBuild in order to build our container image.
* __`cifar10`__ is the directory which contains the files that are installed in the container.

In this simple application, we install only five files in the container. You may only need that many, but if you have many supporting routines, you may wish to install more. These five files show the standard structure of our Python containers, although you are free to choose a different toolset and therefore could have a different layout. If you're writing in a different programming language, you will have a different layout depending on the frameworks and tools you choose.

The files that we put in the container are:

* __`cifar10.py`__ is the program that implements our training algorithm.
* __`resnet_model.py`__ is the program that contains our Resnet model. 
* __`nginx.conf`__ is the configuration file for the nginx front-end. Generally, you should be able to take this file as-is.
* __`serve`__ is the program started when the container is started for hosting. It simply launches nginx and loads your exported model with TensorFlow Serving.
* __`train`__ is the program that is invoked when the container is run for training. Our implementation of this script invokes cifar10.py with our our hyperparameter values retrieved from /opt/ml/input/config/hyperparameters.json. The goal for doing this is to avoid having to modify our training algorithm program.

In summary, the two files you probably want to change for your application are `train` and `serve`.

<a id='the-dockerfile'></a>

### The Dockerfile

The Dockerfile describes the image that we want to build. You can think of it as describing the complete operating system installation of the system that you want to run. A Docker container running is quite a bit lighter than a full operating system, however, because it takes advantage of Linux on the host machine for the basic operations. 

For the Python science stack, we start from an official TensorFlow docker image and run the normal tools to install TensorFlow Serving. Then we add the code that implements our specific algorithm to the container and set up the right environment for it to run under.

Let's look at the Dockerfile for this example.

In [ ]:
!cat container/Dockerfile

<a id='building-the-container'></a>

### Building and registering the container

There are multiple ways in which you can build a Docker container. If you have used Docker before, you might be familiar with the commands `docker build` and `docker push`, for example.

In this lab, we are going to automate the building of our container image using [AWS CodeBuild](https://aws.amazon.com/codebuild/):

    AWS CodeBuild is a fully managed continuous integration service that compiles source code, runs tests, and produces software packages that are ready to deploy. With CodeBuild, you don’t need to provision, manage, and scale your own build servers. CodeBuild scales continuously and processes multiple builds concurrently, so your builds are not left waiting in a queue. You can get started quickly by using prepackaged build environments, or you can create custom build environments that use your own build tools. With CodeBuild, you are charged by the minute for the compute resources you use.

In order to leverage CodeBuild to build our Docker container image, we will make use of the provided `buildspec.yml` file that defines the various steps that will be executed during the build process. Let's take a look at the buildspec.yml file:

In [ ]:
!cat container/buildspec.yml

<br/>

In the above file, you see that we have defined 3 build phases:

1. *pre_build*: this phase ensures that CodeBuild successfully logs into our ECR repository
2. *build*: this phase is used to run the `docker build` command, which will build our container image according to our Dockerfile (see [above](#the-dockerfile))
3. *post_build*: this phase is used to run the `docker push` command, which pushes our new image into our ECR repository

<br/>

Before we can run our CodeBuild job, we first need to make sure that we have an ECR respository in which we will store our Docker images.

The following code looks for an ECR repository in the account you're using and the current default region. If the repository doesn't exist, the script will create it.

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-tf-cifar10-example

cd container

chmod +x cifar10/train
chmod +x cifar10/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

aws ecr describe-repositories --repository-names "${algorithm_name}"

Next, we need to package up our Dockerfile and TensorFlow training script into a ZIP file, and store it in S3.

CodeBuild will later download this ZIP file in order to build the container image that SageMaker will use for training.

In [ ]:
import sagemaker

sm_session = sagemaker.session.Session()
s3_root = sm_session.default_bucket() + "/codebuild_tmp/"

# install zip, as it is sometimes missing
print("Attempting to install zip using apt-get:")
!apt-get install -y zip
print()

# zip the contents of the 'container' directory into "codebuild__tf_cifar10.zip"
print("Zipping contents of ./container/ into codebuild__tf_cifar10.zip:")
!cd container; zip -r ../codebuild__tf_cifar10.zip *; cd ..; echo; ls -alh *.zip  
print()

# copy codebuild__tf_cifar10.zip to a temporary S3 location which we'll need when submitting our CodeBuild job
print("Uploading zip file to S3:")
!aws s3 cp ./codebuild__tf_cifar10.zip s3://{s3_root}

<br/>
<br/>

Now that we have uploaded our training script and Dockerfile to S3, the next step is to define a CodeBuild project. Remember, we'll be using a CodeBuild project to automate the Docker build process.

The following code block will generate a JSON file describing our new CodeBuild project:

In [ ]:
import json
import boto3
boto3_session = boto3.session.Session()

algorithm_name = "sagemaker-tf-cifar10-example"
project_name = algorithm_name + "_01"
region = boto3_session.region_name
account = boto3.client('sts').get_caller_identity().get('Account')
codebuild_role = "arn:aws:iam::" + account + ":role/codebuild__sagemaker_workshop_role"

# Below we define our JSON template that will be passed into the "aws codebuild create-project ..." CLI command.
# This template defines the overall CodeBuild Project, as well as our custom environment variables that control the build process
# NOTE: You do not need to change anything below.

json_template = {
  "name": project_name,
  "source": {
    "type": "S3",
    "location": s3_root + "codebuild__tf_cifar10.zip"
  },
  "artifacts": {
    "type": "NO_ARTIFACTS"
  },
  "environment": {
    "type": "LINUX_CONTAINER",
    "image": "aws/codebuild/standard:4.0",
    "computeType": "BUILD_GENERAL1_SMALL",
    "environmentVariables": [
      {
        "name": "AWS_DEFAULT_REGION",
        "value": region
      },
      {
          "name": "ALGORITHM_NAME",
          "value": algorithm_name
      },
        {
            "name": "IMAGE_NAME",
            "value": account + ".dkr.ecr." + region + ".amazonaws.com/" + algorithm_name + ":latest"
        }
    ],
    "privilegedMode": True
  },
  "serviceRole": codebuild_role
}


# Output the JSON file to 'codebuild_create_project.json' in the notebook directory
with open("codebuild_create_project.json", "w") as f:
    f.write(json.dumps(json_template))

<br/>

Next, let's check the contents of our newly created JSON tempate file:

In [ ]:
import json

with open("codebuild_create_project.json") as f:
    tmp = json.load(f)
    display(Markdown("```json\n" + json.dumps(tmp, indent=4) + "\n```"))

<br/>

Next, we use the AWS CLI to create a new CodeBuild Project, referencing our JSON template file:

In [ ]:
!aws codebuild create-project --cli-input-json file://codebuild_create_project.json

<br/>
<br/>

With our CodeBuild Project in place, we now use the AWS CLI to initiate a build job:

In [ ]:
# This will build the Docker container image defined by our Dockerfile and push the image into ECR
!aws codebuild start-build --project-name=sagemaker-tf-cifar10-example_01

<br/>

Now that you have started the CodeBuild build process, it is a good idea to monitor the progress of your build job.

When you run the following cell, it will create links for the CodeBuild and ECR consoles:

In [ ]:
link1 = "* [CodeBuild](https://{region}.console.aws.amazon.com/codesuite/codebuild/599069043765/projects/sagemaker-tf-cifar10-example_01/)".format(region=region)
link2 = "* [ECR](https://{region}.console.aws.amazon.com/ecr/repositories/sagemaker-tf-cifar10-example/)".format(region=region)
links = link1 + "\n" + link2

display(Markdown("#### Generated Links:\n" + links))

Begin by clicking the `CodeBuild` link above. Within the CodeBuild console, you will see a description of your ongoing build job, along with the associated build status. Monitor your ongoing build job (try clicking the first link under 'Build run' to view the log files). When your container image has been created, the 'Latest Build Status' column will show as `Succeeded`.

Once the Codebuild job is complete, click the `ECR` link above. If your CodeBuild job was successful in pushing your new image into ECR, you will see a new image with the image tag `latest`. Under the `Pushed at` column for this image, you should see that this image was pushed very recently.

If for some reason you do not see your new image within ECR, please double-check that your Codebuild job was successful, and repeat the [Building and registering the container](#building-the-container) section as required.

<br/>
<br/>

<br/>
<br/>

<a id='part2'></a>

# Part 2: Using your Algorithm in Amazon SageMaker
Once you have your container packaged, you can use it to train and serve models.

Let's use our new container in a SageMaker training job.

<a id="set-up-environment"></a>

## Set up the environment
Here we specify the bucket to use and the role that is used for working with SageMaker.

In [ ]:
# S3 prefix
prefix = 'DEMO-tensorflow-cifar10'

<a id='create-the-session'></a>

## Create the session

The session remembers our connection parameters to SageMaker. We use it to perform all of our SageMaker operations.

In [ ]:
import sagemaker as sage

sess = sage.Session()

<a id='upload-data-for-training'></a>

## Upload the data for training

Our training algorithm is expecting our training data to be in the file format of [TFRecords](https://www.tensorflow.org/guide/datasets), which is a simple record-oriented binary format that many TensorFlow applications use for training data.

Below is a Python script adapted from the [official TensorFlow CIFAR-10 example](https://github.com/tensorflow/models/tree/master/tutorials/image/cifar10_estimator), which downloads the CIFAR-10 dataset and converts them into TFRecords.

Once we've downloaded and converted the CIFAR-10 dataset, We will then use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [ ]:
!pip install --quiet ipywidgets
!python3 utils/generate_cifar10_tfrecords.py --data-dir=/tmp/cifar-10-data

In [ ]:
# There should be three tfrecords. (eval, train, validation)
!ls -alh /tmp/cifar-10-data

In [ ]:
# Now we upload the CIFAR-10 data to a known location in S3

WORK_DIRECTORY = '/tmp/cifar-10-data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

<a id='create-and-fit-estimator'></a>

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [ ]:
import boto3
from sagemaker import get_execution_role

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name
role = get_execution_role()

algorithm_name = 'sagemaker-tf-cifar10-example'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

In [ ]:
from sagemaker.estimator import Estimator

hyperparameters = {'train-steps': 100}

instance_type = 'ml.m4.xlarge'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters)

estimator.fit(data_location)

<br/>
<br/>

Lastly, let's see where our model artifacts were stored in S3:

In [ ]:
estimator.model_data